# Collaborativ Filtering

Ziel ist, eine Tabelle zu haben, mit verschiedenen Playlisten in den Zeilen und Songs in den Spalten. Die Zellen sagen aus, ob ein Song in einer Playlist ist oder nicht (0 oder 1).

Die Tabelle kann dann für Collab Filtering verwendet werden.

Da unsere Datnebank in einem anderen Netzwerk liegt, wird zunächst ein Tunnel über ein Terminal benötigt:

```ssh <username>@login1.mi.hs-rm.de -L 9001:db.intern.mi.hs-rm.de:5432```

In [1]:
import psycopg2
import pandas as pd
from getpass import getpass
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

## Generate Collab Filtering Table

Also brauchen wir zunächst einmal die Tabelle mit dem Inhalt, welches Lied in welcher Playlist ist.

### Datenbankverbindung

In [2]:
user = input("Bitte User für DB eingeben")
pswd = getpass("Bitte User für DB eingeben")
conn = psycopg2.connect(f"dbname='orent001_spotify_test' user='{user}' host='localhost' port='9001' password='{pswd}'")

In [3]:
# query = "SELECT playlist.name AS playlist, song.track_name AS song FROM playlist INNER JOIN p_enthaelt_s ON p_enthaelt_s.playlist = playlist.playlist_id INNER JOIN song ON song.song_id = p_enthaelt_s.song;"
# playlists = pd.io.sql.read_sql_query(query,conn)
# playlists

### Select Data from SQL and pivot it to a collab usefull table

pd.pivot_table macht aus:           gleich:

```
playlist | song | drinnen               song | A | B | C 
---------+------+--------           playlist |   |   |   
 0       | A    | 1                 ---------+---+---+---
 0       | B    | 1          ==>     0       | 1 | 1 | 1 
 0       | C    | 1                  1       | 0 | 1 | 0 
 1       | B    | 1                  2       | 0 | 1 | 1 
 2       | B    | 1      
 2       | C    | 1      
```

Dann werden die Column Namen entwernt und die Werte in der Tabelle standartisiert und 

dauert mit den Testdaten ca. 5 Minuten

In [17]:
query = "SELECT playlist, song, 1 as drinnen FROM p_enthaelt_s;"
songs = pd.io.sql.read_sql_query(query,conn)
table = pd.pivot_table(songs, values='drinnen', columns='song', index='playlist').fillna(0)
table.columns.name = None

def standardize(row):
    new_row = (row - row.mean())/1
    return new_row

ratings_std = table.apply(standardize)
ratings_std

MemoryError: Unable to allocate 4.02 GiB for an array with shape (5000, 107781) and data type float64

In [15]:
item_similarity = cosine_similarity(table.T)
item_similarity_df = pd.DataFrame(item_similarity, table.columns, table.columns)
item_similarity_df

MemoryError: Unable to allocate 86.6 GiB for an array with shape (107781, 107781) and data type float64

In [7]:
table.to_csv(index=False)